# Tiny few shot classifer

Im folgenden wird ein few shot classifier für multilabels im [SetFit-Frame](https://huggingface.co/blog/setfit) Format erzeugt. Gearbeitet wird mit `Python 3.12.3`.

# Pakete laden

In einem ersten Schritt werden die nötigen Pakete geladen. 

In [4]:
from datasets import load_dataset  # Zum Laden von Datensätzen aus der Hugging Face `datasets` Bibliothek
from sentence_transformers.losses import CosineSimilarityLoss  # Verlustfunktion basierend auf der Kosinusähnlichkeit, für Aufgaben wie Textähnlichkeit

from setfit import SetFitModel, SetFitTrainer  # Zum Laden des SetFit-Modells und Trainers, speziell für Few-Shot Learning mit Satztransformern
import pandas as pd  # Pandas für Datenmanipulation und -analyse
from sklearn.preprocessing import LabelEncoder  # Zum Encoden von Labels in numerische Werte für maschinelles Lernen
import os  # Für den Umgang mit Betriebssystemfunktionen (z.B. Dateipfade)
from datasets import Dataset, DatasetDict  # Zum Erstellen und Verwalten von Datensätzen in Hugging Face `datasets`-Format
from sklearn.model_selection import train_test_split  # Zum Aufteilen der Daten in Trainings- und Testsets

from transformers import TrainingArguments  # Zum Festlegen von Trainingsparametern für Modelle in der Transformers-Bibliothek

import torch  # Ermöglicht die GPU-Nutzung und effiziente Verarbeitung von Tensoren
import numpy as np  # Für numerische Berechnungen und Arbeiten mit Arrays

# Arbeitsverzeichnis

Es wird weiterhin das Rootverzeichnis bestimmt.

In [5]:
# checke home-verzeichnis
print(os.getcwd())
# setze home-verzeichnis
os.chdir('c:/Users/Hueck/OneDrive/Dokumente/GitHub/future_skill_classification/')

c:\Users\Hueck\OneDrive\Dokumente\GitHub\future_skill_classification


# Daten

# Daten laden

Das zur Zeit keine Daten für das trainieren eines multilable Classifiers vorliegen, werden die Trainings-Daten genutzt, die ursprünglich von Franziska Weber und Felix Süßenbach erzeugt wurden.

In [6]:
# Laden der Excel-Datei
pd_train_data = pd.read_excel('data/train_data_franziska.xlsx')

# Für die weitere Verarbeitung werden die Daten ins  Hugging Face Dataset-Format transformiert. 

# /////////////////////////////////////////////////
# Hinweis: Das Dataset-Format der Hugging Face datasets-Bibliothek basiert auf Apache Arrow und ist speichereffizient, was schnelles Laden und Verarbeiten großer Datensätze ermöglicht. Es integriert sich nahtlos in Machine-Learning-Workflows und unterstützt einfache Transformationen und Vorverarbeitung durch Methoden wie .map(). Zudem ist es flexibel und kompatibel mit Pandas DataFrames sowie gängigen Frameworks wie PyTorch und TensorFlow.
#//////////////////////////////////////////////////

train_data = Dataset.from_pandas(pd_train_data)
print(train_data)
 

Dataset({
    features: ['sentence', 'Data Analytics & KI', 'Softwareentwicklung', 'Nutzerzentriertes Design', 'IT-Architektur', 'Hardware/Robotikentwicklung', 'Quantencomputing', 'Digital Literacy', 'Digital Ethics', 'Digitale Kollaboration', 'Digital Learning', 'Agiles Arbeiten', 'Lösungsfähigkeit', 'Kreativität', 'Unternehmerisches Handeln & Eigeninitiative', 'Interkulturelle Kommunikation', 'Resilienz', 'Urteilsfähigkeit', 'Innovationskompetenz', 'Missionsorientierung', 'Veränderungskompetenz', 'Dialog- und Konfliktfähigkeit'],
    num_rows: 1577
})


## Daten verarbeiten

In einem ersten Schritt werden die Features des Classifiers ausgewählt. Da sich insbesondere *klassische* und *transformative Future Skills* als schwer nachvollziehbar in den Daten erwiesen haben, werden diese aus dem Classifier ausgeschlossen, ebenso wie die Kurstitel und -beschreibungen (`sentence`). 

In [11]:
# ziehe column names als Liste
features = train_data.column_names
type(features)

# definiere skills, die nicht vom classifier berücksichtigt werden
to_remove = ['sentence','Lösungsfähigkeit', 'Kreativität', 'Unternehmerisches Handeln & Eigeninitiative', 
             'Interkulturelle Kommunikation', 'Resilienz', 'Urteilsfähigkeit', 
             'Innovationskompetenz', 'Missionsorientierung', 'Veränderungskompetenz', 
             'Dialog- und Konfliktfähigkeit']

# schließe die oben definierten Variablen aus der Feature-Liste aus
filtered_features = [feature for feature in features if feature not in to_remove]

features = filtered_features
print(features)

['Data Analytics & KI', 'Softwareentwicklung', 'Nutzerzentriertes Design', 'IT-Architektur', 'Hardware/Robotikentwicklung', 'Quantencomputing', 'Digital Literacy', 'Digital Ethics', 'Digitale Kollaboration', 'Digital Learning', 'Agiles Arbeiten']


Für das Trainieren des Classifiers wird im folgenden ein *One-Hot-Encoding*-Array der Features erzeugt. Dies ist deshalb nötig, da im folgenden ein Modell für ein Multi-Label-Szenario trainiert werden soll: Ein Kursangebot kann den erwerb mehrerer Future Skills versprechen, etwa Softwareentwicklung **&** IT-Architektur. Das One-Hot-Encoding wird benötigt, um die Labels der Features in eine Form zu bringen, die das Classifier Modell von SetFit korrekt verarbeiten kann. Es stellt sicher, dass alle Klassen gleichwertig behandelt werden, und ermöglicht die Berechnung von Ähnlichkeiten verschiedener Kursinhalte.

Um den *One-Hot-Encoding*-Array zu erzeugen definieren wir `encode_labels(record)`: Diese Funktion geht durch jede Zeile eines Data Frames und erstellt eine neue Spalte namens `labels`. In dieser Spalte werden die Werte aus den oben definierten "features" gesammelt und in eine Liste gepackt. Diese Liste enthält Werte der Features.

    Beispiel: Mit Blick auf die Features 'Data Analytics & KI', 'Softwareentwicklung' und 'Nutzerzentriertes Design', könnte ein Zeilenvektor des Arrays eines Kurses der sich mit Data Analytics & KI / Softwareentwicklung beschäftigt so aussehen: 

    `[1, 1, 0]`

In [12]:
def encode_labels(record):
    return {"labels": [record[feature] for feature in features]}


dataset = train_data.map(encode_labels)
pd_dataset = dataset.to_pandas()

c:\Users\Hueck\AppData\Local\Programs\Python\Python312\Lib\site-packages\datasets\utils\_dill.py:379: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
  obj.co_lnotab,  # for < python 3.10 [not counted in args]


Map:   0%|          | 0/1577 [00:00<?, ? examples/s]

sample 8 data points

In [7]:
num_samples = 8
samples = np.concatenate(
    [np.random.choice(np.where(dataset[f])[0], num_samples) for f in features]
)

In [12]:
train_dataset = dataset.select(samples)
eval_dataset = dataset.select(
    np.setdiff1d(np.arange(len(dataset)), samples)
)
print(eval_dataset)

Dataset({
    features: ['sentence', 'Data Analytics & KI', 'Softwareentwicklung', 'Nutzerzentriertes Design', 'IT-Architektur', 'Hardware/Robotikentwicklung', 'Quantencomputing', 'Digital Literacy', 'Digital Ethics', 'Digitale Kollaboration', 'Digital Learning', 'Agiles Arbeiten', 'Lösungsfähigkeit', 'Kreativität', 'Unternehmerisches Handeln & Eigeninitiative', 'Interkulturelle Kommunikation', 'Resilienz', 'Urteilsfähigkeit', 'Innovationskompetenz', 'Missionsorientierung', 'Veränderungskompetenz', 'Dialog- und Konfliktfähigkeit', 'labels'],
    num_rows: 1511
})


In [10]:
from setfit import SetFitModel

model = SetFitModel.from_pretrained(model_id, multi_target_strategy="one-vs-rest")

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [13]:
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitTrainer

trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss_class=CosineSimilarityLoss,
    num_iterations=20,
    column_mapping={"sentence": "text", "labels": "label"},
)

C:\Users\Hueck\AppData\Local\Temp\ipykernel_8656\2320776502.py:4: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset
c:\Users\Hueck\AppData\Local\Programs\Python\Python312\Lib\site-packages\datasets\utils\_dill.py:379: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
  obj.co_lnotab,  # for < python 3.10 [not counted in args]


Map:   0%|          | 0/88 [00:00<?, ? examples/s]

In [14]:
trainer.train()


***** Running training *****
  Num unique pairs = 3520
  Batch size = 16
  Num epochs = 1


  0%|          | 0/220 [00:00<?, ?it/s]

{'embedding_loss': 0.259, 'grad_norm': 1.792460560798645, 'learning_rate': 9.090909090909091e-07, 'epoch': 0.0}
{'embedding_loss': 0.1768, 'grad_norm': 1.3233823776245117, 'learning_rate': 1.7171717171717173e-05, 'epoch': 0.23}
{'embedding_loss': 0.0879, 'grad_norm': 1.1306359767913818, 'learning_rate': 1.2121212121212122e-05, 'epoch': 0.45}
{'embedding_loss': 0.0574, 'grad_norm': 0.8585573434829712, 'learning_rate': 7.070707070707071e-06, 'epoch': 0.68}
{'embedding_loss': 0.0404, 'grad_norm': 1.0427318811416626, 'learning_rate': 2.02020202020202e-06, 'epoch': 0.91}


Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

{'train_runtime': 72.8705, 'train_samples_per_second': 48.305, 'train_steps_per_second': 3.019, 'train_loss': 0.08737015371972864, 'epoch': 1.0}


In [15]:
trainer.push_to_hub("Chernoffface/fs-setfit-multilable-model")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

model_head.pkl:   0%|          | 0.00/72.2k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Chernoffface/fs-setfit-multilable-model/commit/8a4eb18f415b13e1ec16085f1c5c6af7a8c09387', commit_message='Add SetFit model', commit_description='', oid='8a4eb18f415b13e1ec16085f1c5c6af7a8c09387', pr_url=None, pr_revision=None, pr_num=None)

In [17]:
from setfit import SetFitModel

model = SetFitModel.from_pretrained("Chernoffface/fs-setfit-multilable-model")

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

c:\Users\Hueck\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Hueck\.cache\huggingface\hub\models--Chernoffface--fs-setfit-multilable-model. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


modules.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/209 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.39k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/305 [00:00<?, ?B/s]

config_setfit.json:   0%|          | 0.00/56.0 [00:00<?, ?B/s]

model_head.pkl:   0%|          | 0.00/72.2k [00:00<?, ?B/s]

In [25]:
preds = model(
    [
        "Im Rahmen des Praktikums werden die bereits erlernten Grundlagen der Java Programmierung von Modul IN1501 erweitert und vertieft. Hierbei steht dessen praktischer Bezug im Vordergrund. .",
        "Aim/ learning outcomes:• learning of the programming language C and understanding of basic concepts of programming• finding and correcting programming errors• development of computer programs and organization of complex projects• working with software libraries• independent analysis of scientific problems and their implementation in CContent:Linux basics, the C++ programming language (e.g. data types, loops, functions, classes, templates), compiler (function, process), OpenSource tools (e.g. make, gnuplot), implementation of numerical algorithms as application examples"
    ]
)
[[f for f, p in zip(features, ps) if p] for ps in preds]

 

[['Softwareentwicklung'], ['Softwareentwicklung']]